In [ ]:
pip install linearmodels

In [2]:
import numpy as np
import pandas as pd
from linearmodels.panel import PanelOLS

In [ ]:
dataset = pd.read_csv('/content/Glyphosate Suicide Dataset.csv')

In [ ]:
df = dataset.copy()
df.columns = (df.columns.str.strip().str.replace(r"[^\w]+","_", regex=True).str.replace(r"_$","", regex=True))
df["County_Code"] = df["County_Code"].astype(str)
df["Year"] = pd.to_numeric(df["Year"], errors="coerce").astype("Int64")
df = df[df['Unemployment_Rate']!='N.A.'].reset_index(drop=True)
df["glyph_q"] = (df.groupby("Year")["Glyphosate_kg_per_km2"].transform(lambda x: pd.qcut(x, 5, labels=["Q1","Q2","Q3","Q4","Q5"], duplicates="drop")))
df["glyph_q_ord"] = df["glyph_q"].astype(str).str.replace("Q", "", regex=False)
df["glyph_q_ord"] = pd.to_numeric(df["glyph_q_ord"], errors="coerce")

OUTCOMES = {"all": "Suicide_Death_Rate", "firearm": "Firearm_Suicide_Death_Rate", "nonfirearm": "Non_Firearm_Suicide_Death_Rate",}

base_covars = ["Mean_Age", "Female", "American_Indian_or_Alaska_Native", "Asian_or_Pacific_Islander", "Black_or_African_American", "Hispanic_or_Latino", "Total_Alcohol", "Unemployment_Rate", "PM2_5", "Urbanicity",]

def _build_X(d, use_trend=False):
    # exposure: either quintile dummies (Q1 ref) or ordinal trend term
    if use_trend:
        X = pd.DataFrame({"glyph_trend": d["glyph_q_ord"].astype(float)}, index=d.index)
    else:
        X = pd.get_dummies(d["glyph_q"], prefix="glyph", drop_first=True)  # glyph_Q2..glyph_Q5

    # numeric covariates
    X = pd.concat([X, d[["Mean_Age","Female", "American_Indian_or_Alaska_Native","Asian_or_Pacific_Islander","Black_or_African_American", "Hispanic_or_Latino", "Total_Alcohol","Unemployment_Rate","PM2_5"]]], axis=1)

    # urbanicity dummies
    X = pd.concat([X, pd.get_dummies(d["Urbanicity"].astype(str), prefix="Urban", drop_first=True)], axis=1)
    return X.loc[:, X.nunique(dropna=True) > 1]

def fit_panel_loglinear(rate_col, eps=1e-9, trend=False):
    d = df.copy()
    d[rate_col] = pd.to_numeric(d[rate_col], errors="coerce")
    d["Population"] = pd.to_numeric(d["Population"], errors="coerce")

    req = ["County_Code","Year","Population",rate_col] + base_covars + (["glyph_q_ord"] if trend else ["glyph_q"])
    d = d.dropna(subset=req).copy()

    d["y"] = np.log(d[rate_col].astype(float) + eps)
    d = d.set_index(["County_Code","Year"]).sort_index()

    X = _build_X(d, use_trend=trend)

    mod = PanelOLS(
        d["y"], X,
        entity_effects=True,
        time_effects=True,
        weights=d["Population"],
        drop_absorbed=True
    )
    return mod.fit(cov_type="clustered", cluster_entity=True)

def pct_change_table(res):
    params, ses, pvals = res.params, res.std_errors, res.pvalues
    q_terms = [t for t in params.index if t.startswith("glyph_")]  # glyph_Q2..glyph_Q5
    out = pd.DataFrame({"beta": params[q_terms], "pct": 100*(np.exp(params[q_terms]) - 1), "pct_lo": 100*(np.exp(params[q_terms] - 1.96*ses[q_terms]) - 1), "pct_hi": 100*(np.exp(params[q_terms] + 1.96*ses[q_terms]) - 1), "p": pvals[q_terms],})
    out.index = out.index.str.replace("glyph_", "", regex=False)  # Q2..Q5
    return out.sort_index()

def ptrend_from_res(res_trend):
    # p-trend is the p-value of the single ordinal term
    term = "glyph_trend"
    return float(res_trend.pvalues[term]) if term in res_trend.pvalues.index else np.nan

# --- run outcomes: main model + p-trend model ---
results, tables, ptrend = {}, {}, {}

for k, col in OUTCOMES.items():
    res = fit_panel_loglinear(col, trend=False)
    res_trend = fit_panel_loglinear(col, trend=True)

    results[k] = res
    tables[k] = pct_change_table(res)
    ptrend[k] = ptrend_from_res(res_trend)

    print("\n===============================")
    print(f"Outcome: {k} ({col})")
    print(res.summary)
    print("\nQuintile % change vs Q1:")
    print(tables[k])
    print(f"\nP-trend (ordinal Q1–Q5): {ptrend[k]:.6g}")